In [2]:
from utils import evaluate_policy, str2bool
from datetime import datetime
from PPO import PPO_discrete
import  gym
import os, shutil
import argparse
import torch


In [3]:
'''Hyperparameter Setting'''
parser = argparse.ArgumentParser()
parser.add_argument('--dvc', type=str, default='cuda', help='running device: cuda or cpu')
parser.add_argument('--EnvIdex', type=int, default=1, help='CP-v1, LLd-v2')
parser.add_argument('--write', type=str2bool, default=False, help='Use SummaryWriter to record the training')
parser.add_argument('--render', type=str2bool, default=True, help='Render or Not')
parser.add_argument('--Loadmodel', type=str2bool, default=False, help='Load pretrained model or Not')
parser.add_argument('--ModelIdex', type=int, default=200000, help='which model to load')

parser.add_argument('--seed', type=int, default=209, help='random seed')
parser.add_argument('--T_horizon', type=int, default=2048, help='lenth of long trajectory')
parser.add_argument('--Max_train_steps', type=int, default=5e7, help='Max training steps')
parser.add_argument('--save_interval', type=int, default=1e5, help='Model saving interval, in steps.')
parser.add_argument('--eval_interval', type=int, default=5e3, help='Model evaluating interval, in steps.')

parser.add_argument('--gamma', type=float, default=0.99, help='Discounted Factor')
parser.add_argument('--lambd', type=float, default=0.95, help='GAE Factor')
parser.add_argument('--clip_rate', type=float, default=0.2, help='PPO Clip rate')
parser.add_argument('--K_epochs', type=int, default=10, help='PPO update times')
parser.add_argument('--net_width', type=int, default=64, help='Hidden net width')
parser.add_argument('--lr', type=float, default=1e-4, help='Learning rate')
parser.add_argument('--l2_reg', type=float, default=0, help='L2 regulization coefficient for Critic')
parser.add_argument('--batch_size', type=int, default=64, help='lenth of sliced trajectory')
parser.add_argument('--entropy_coef', type=float, default=0, help='Entropy coefficient of Actor')
parser.add_argument('--entropy_coef_decay', type=float, default=0.99, help='Decay rate of entropy_coef')
parser.add_argument('--adv_normalization', type=str2bool, default=False, help='Advantage normalization')
opt = parser.parse_args([])
opt.dvc = torch.device(opt.dvc) # from str to torch.device
print(opt)

Namespace(dvc=device(type='cuda'), EnvIdex=1, write=False, render=True, Loadmodel=False, ModelIdex=200000, seed=209, T_horizon=2048, Max_train_steps=50000000.0, save_interval=100000.0, eval_interval=5000.0, gamma=0.99, lambd=0.95, clip_rate=0.2, K_epochs=10, net_width=64, lr=0.0001, l2_reg=0, batch_size=64, entropy_coef=0, entropy_coef_decay=0.99, adv_normalization=False)


In [4]:
EnvName = ['CartPole-v1','LunarLander-v2']
BriefEnvName = ['CP-v1','LLd-v2']

In [5]:
env = gym.make(EnvName[opt.EnvIdex], new_step_api=True,render_mode = "human" if opt.render else None)

In [6]:
opt.state_dim = env.observation_space.shape[0]
opt.action_dim = env.action_space.n
opt.max_e_steps = env._max_episode_steps

# Seed Everything
env_seed = opt.seed
torch.manual_seed(opt.seed)
torch.cuda.manual_seed(opt.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print("Random Seed: {}".format(opt.seed))

print('Env:',BriefEnvName[opt.EnvIdex],'  state_dim:',opt.state_dim,'  action_dim:',opt.action_dim,'   Random Seed:',opt.seed, '  max_e_steps:',opt.max_e_steps)
print('\n')


Random Seed: 209
Env: LLd-v2   state_dim: 8   action_dim: 4    Random Seed: 209   max_e_steps: 1000




In [7]:
agent = PPO_discrete(**vars(opt))

In [8]:
agent.load(EnvName[opt.EnvIdex],opt.ModelIdex)

In [9]:
while True:
    s = env.reset()
    done = False
    while not done:
        # Take deterministic actions at test time
        a, logprob_a = agent.select_action(s, deterministic=False)
        s_next, r, dw, tr, info = env.step(a)
        done = (dw or tr)
        s = s_next
        
    


KeyboardInterrupt: 

In [10]:
env.close()